In [1]:
!pip install torch torchaudio transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2Model
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import soundfile as sf
import torchaudio

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
# Load Common Voice dataset
dataset = load_dataset("mozilla-foundation/common_voice_13_0", "ar", split="validation+train+test")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9782it [00:00, 97794.21it/s]
Reading metadata...: 28167it [00:00, 85958.52it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10409it [00:00, 115365.03it/s]


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10445it [00:00, 81193.52it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11011it [00:00, 110100.13it/s]
Reading metadata...: 36192it [00:00, 129236.14it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14994it [00:00, 115959.99it/s]


In [8]:
print(dataset)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 49021
})


In [9]:
# Removing unwanted features (we only want audio and its transcription)
dataset = dataset.remove_columns(["client_id", "path", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"])

print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 49021
})


In [10]:
print(dataset[0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/d68675862a8f904b8c115f046d22a9eb133d1b8d146881ef631041738b0d8f3e/ar_dev_0/common_voice_ar_24079904.mp3', 'array': array([-3.69482223e-13,  6.82121026e-13,  1.42108547e-12, ...,
       -8.46983676e-06, -1.07555734e-05, -5.13326540e-06]), 'sampling_rate': 48000}, 'sentence': 'وَقْفَةُ الأهرامِ فيما بَيْنَنَا لِصُروفِ الدَّهرِ وَقْفَتى أنا'}


In [11]:
vocabulary = ['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ء', 'آ', 'أ', 'إ', 'ؤ', 'ئ', 'ة', 'ى', 'ﻻ', 'ﻷ', 'ﻹ', 'ﻵ',' ', '.']

In [12]:
vocabulary_set = set(vocabulary)  # Convert your existing vocabulary list to a set

# Initialize an empty set for characters from the dataset
dataset_characters = set()

for data_item in dataset:
    sentence = data_item['sentence']
    dataset_characters.update(sentence)

# Perform a union of the dataset characters with the existing vocabulary set
combined_vocabulary = vocabulary_set.union(dataset_characters)

# If you need the combined vocabulary as a list
combined_vocabulary_list = list(combined_vocabulary)

In [13]:
print(len(combined_vocabulary_list))

126


In [14]:
print(combined_vocabulary_list)

['u', 'ﻻ', 'ب', 's', 'ص', 'A', 'ق', 'x', '!', ';', 'ٰ', 'و', 'ی', 'ا', 'w', 'b', '(', 'ج', '.', 'V', 'ي', 'ٌ', 'ٍ', 'ف', 'y', 'Q', '؛', ' ', 'ڨ', '…', 'ۚ', 'ى', 'ئ', 'ش', 'ط', 'ۗ', 'a', 'v', 'P', 'X', 'ً', 'T', 'g', 'س', 'خ', 'ﻹ', 'ُ', '“', 'E', 'd', 'ﺃ', '«', '?', 'M', '،', 'ھ', 'ع', '؟', 'K', 'چ', 'C', 'p', 'ر', '—', '☭', 'e', 'ح', 'ﻵ', '»', '-', 'ۖ', 'D', ')', ':', 'غ', "'", 'k', 'ﻷ', 'f', 'j', 'ِ', 'r', 'B', 'ت', 'ه', 'إ', 'ؤ', 'ذ', 'ة', 't', 'ْ', 'U', 'ظ', 'آ', 'ن', 'm', 'H', '_', 'ۘ', 'أ', '"', 'i', 'م', '”', 'ل', 'l', 'ك', 'G', 'د', 'h', 'ک', 'ز', 'n', ',', 'ـ', 'S', 'ث', 'c', 'I', 'ۛ', 'ء', 'َ', 'ض', 'R', 'o', 'ّ']


In [15]:
class CustomProcessor:
    def __init__(self, vocab):
        self.vocab = vocab
        self.char_to_index = {char: index for index, char in enumerate(vocab)}

    def text_to_int(self, text):
        return [self.char_to_index[char] for char in text]

    def int_to_text(self, indices):
        return ''.join([self.vocab[index] for index in indices])

In [16]:
object_voc = CustomProcessor(combined_vocabulary_list)

In [17]:
text = "مرحبا"
encoding = object_voc.text_to_int(text)
decoding = object_voc.int_to_text(encoding)
print(encoding)
print(decoding)

[102, 62, 66, 2, 13]
مرحبا


In [18]:
import torch
from torch.utils.data import Dataset
import torchaudio

In [19]:
class CommonVoiceDataset(Dataset):
    def __init__(self, dataset, vocab):
        self.dataset = dataset
        self.processor = CustomProcessor(vocab)
        self.spectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Load and resample audio
        audio_input, sampling_rate = torchaudio.load(self.dataset[idx]["audio"]["path"])
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            audio_input = resampler(audio_input)

        # Convert to spectrogram
        # spectrogram = self.spectrogram_transform(audio_input).squeeze(0)
        spectrogram = self.spectrogram_transform(audio_input)
        # Normalize spectrogram
        spectrogram = (spectrogram - spectrogram.mean()) / spectrogram.std()

        # Process text
        sentence = self.dataset[idx]["sentence"]

        labels = self.processor.text_to_int(sentence)
        labels = torch.tensor(labels)

        return spectrogram, labels, spectrogram.shape[-1]

In [20]:
# Instantiate the dataset
common_voice_dataset = CommonVoiceDataset(dataset, combined_vocabulary_list)

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [21]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [38]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

In [39]:
import torch
import torch.nn as nn

class CustomSTTModel2(nn.Module):
    def __init__(self, num_classes):
        super(CustomSTTModel2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.pos_encoder = PositionalEncoding(d_model=2048)  # Set d_model to match the input size of GRU1

        self.gru1 = nn.GRU(input_size=2048, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(input_size=256, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.5)

        self.fc = nn.Linear(256, 256)
        self.out = nn.Linear(256, num_classes + 1)  # num_classes includes the Arabic charset + 1 for CTC blank

    def forward(self, x):
        # print("cccccccccccccccccccccccccc")
        # x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool(x)
        # print("dddddddddddddddddddddddddddd")
        # x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool(x)
        # print(x.shape)
        # torch.Size([8, 64, 32, 166]) [Batch Size, Channels, Height(frequency), Width(time)]
        # print("eeeeeeeeeeeeeeeeeeeeeeeeeee")
        x = x.permute(0, 3, 1, 2)  # Rearrange dimensions for GRU; as sequence models expect [Sequence Length (Time Steps),  Batch Size, Feature Size (Number of Features per Time Step)]
        # This often involves flattening the non-time dimensions (like channels and frequency bins) into a single feature dimension
        # print(x.shape)
        # torch.Size([8, 166, 64, 32]) [Batch Size, Width, Channels, Height]
        x = torch.flatten(x, start_dim=2)  # Flatten the convolutional features
        # print("fffffffffffffffffffffffffff")
        # print(x.shape)
        # torch.Size([8, 166, 2048]) [Batch Size, Width, Channels * Height]
        # Apply positional encoding
        x = self.pos_encoder(x)

        x, _ = self.gru1(x)
        # print(x.shape)
        # print("gggggggggggggggggggggggggggg")
        x = self.dropout(x)

        x, _ = self.gru2(x)
        # print(x.shape)
        # print("hhhhhhhhhhhhhhhhhhhhhhhhhhhh")
        x = self.dropout(x)


        # x = torch.relu(self.fc(x))
        x = self.fc(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.out(x)
        # print(x.shape)
        # print("iiiiiiiiiiiiiiiiiiiiiiiii")
        return x

In [40]:
# Instantiate the model
custom_model_2 = CustomSTTModel2(len(combined_vocabulary_list))

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Separate the data items
    spectrograms, labels, input_lengths = zip(*batch)

    spectrograms = [s.permute(2, 0, 1) for s in spectrograms]  # Shape: [time_steps, channels, n_mels] since we need to pad the time_steps and pad_sequence pads the first dimension

    # Pad the spectrograms to have the same time length
    # Note: pad_sequence expects a list of tensors, padding them to match the longest tensor
    spectrograms = pad_sequence(spectrograms, batch_first=True, padding_value=0)

    # Permute back to original dimension order after padding
    spectrograms = spectrograms.permute(0, 2, 3, 1)  # Shape: [batch, channels, n_mels, time_steps]

    # Concatenate all labels lengths
    label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long, device=device)

    # Concatenate all labels (no need for padding)
    labels = torch.cat(labels)

    # Convert input_lengths into a tensor
    input_lengths = torch.tensor(input_lengths)

    return spectrograms, labels, input_lengths, label_lengths

In [43]:
# Create a DataLoader
batch_size = 8
train_loader = DataLoader(common_voice_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [44]:
best_loss = 10000

def train(model, data_loader, criterion, optimizer, epochs, device):
  global best_loss
  model.to(device)
  for epoch in range(epochs):
      model.train()  # Set the model to training mode
      total_loss = 0
      for batch_idx, (spectrograms, labels, input_lengths, label_lengths) in enumerate(train_loader):
          spectrograms, labels = spectrograms.to(device), labels.to(device)
          # Forward pass
          outputs = model(spectrograms) # [batch, output sequence length, classes]
          output_lengths = outputs.shape[1]
          output_lengths = torch.full((outputs.shape[0],), output_lengths, dtype=torch.int64)
          # print(outputs)
          log_probs = torch.nn.functional.log_softmax(outputs, dim=2) # CTC expects log softmax probabilities
          # print(log_probs.shape)

          # The output of the network needs to be in the shape (output sequence length, batch, classes)
          log_probs = log_probs.permute(1, 0, 2)
          # print(log_probs.shape)
          log_probs = log_probs.to(device)
          label_lengths = label_lengths.to(device)
          output_lengths = output_lengths.to(device)
          # print(output_lengths)
          # print(log_probs.shape)
          # print(label_lengths)
          # print(labels.shape)
          # Calculate loss
          loss = criterion(log_probs, labels, output_lengths, label_lengths)
          if torch.isinf(loss): # This is due to a few bad anomalies in the dataset. Removing these anomalies would make this condition irrelevant
            # print("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")
            # print(output_lengths)
            # print(log_probs.shape)
            # print(label_lengths)
            # print(labels.shape)
            # print(outputs)
            # print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
            # print(log_probs)
            continue

          total_loss += loss.item()

          # Backward pass and optimization
          optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
          optimizer.step()

          # Print progress
          print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item():.4f}")

      # Compute average loss for the epoch
      avg_loss = total_loss / len(data_loader)

      # Save the model every two epochs
      if epoch % 2 == 0:
          torch.save(model.state_dict(), f'model_state_dict_epoch_{epoch}.pth')

      # Update the best model if current model is better
      if avg_loss < best_loss:
          best_loss = avg_loss
          torch.save(model.state_dict(), 'best_model_state_dict.pth')

In [45]:
from torch import nn
import torch.optim as optim

# Define the Loss Function (Criterion)
# The CTC loss function expects logits as inputs, so ensure your model outputs logits
criterion = nn.CTCLoss(blank=len(combined_vocabulary_list)).to(device)

In [46]:
# Define the Optimizer
optimizer = optim.Adam(custom_model_2.parameters(), lr=0.001)

In [47]:
# Run the training
epochs = 10  # Define the number of epochs
train(custom_model_2, train_loader, criterion, optimizer, epochs, device)

Streaming output truncated to the last 5000 lines.
Epoch [1/10], Batch [2951/6128], Loss: 3.2235
Epoch [1/10], Batch [2952/6128], Loss: 3.4484
Epoch [1/10], Batch [2953/6128], Loss: 3.3008
Epoch [1/10], Batch [2954/6128], Loss: 3.2999
Epoch [1/10], Batch [2955/6128], Loss: 3.2316
Epoch [1/10], Batch [2956/6128], Loss: 3.2047
Epoch [1/10], Batch [2957/6128], Loss: 3.3712
Epoch [1/10], Batch [2958/6128], Loss: 3.1626
Epoch [1/10], Batch [2959/6128], Loss: 3.1024
Epoch [1/10], Batch [2960/6128], Loss: 3.1327
Epoch [1/10], Batch [2961/6128], Loss: 3.4530
Epoch [1/10], Batch [2962/6128], Loss: 3.4191
Epoch [1/10], Batch [2963/6128], Loss: 3.4291
Epoch [1/10], Batch [2964/6128], Loss: 3.2661
Epoch [1/10], Batch [2965/6128], Loss: 3.2758
Epoch [1/10], Batch [2966/6128], Loss: 3.3075
Epoch [1/10], Batch [2967/6128], Loss: 3.0757
Epoch [1/10], Batch [2968/6128], Loss: 3.3683
Epoch [1/10], Batch [2969/6128], Loss: 3.3304
Epoch [1/10], Batch [2970/6128], Loss: 3.4226
Epoch [1/10], Batch [2971/612

KeyboardInterrupt: ignored

In [30]:
# Define a function to load an audio file and transform it to a spectrogram
def transform_audio_to_spectrogram(audio_path, transform):
    waveform, sample_rate = torchaudio.load(audio_path)

    # Convert stereo to mono by averaging the two channels if necessary
    if waveform.shape[0] == 2:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    spectrogram = transform(waveform)
    spectrogram = (spectrogram - spectrogram.mean()) / spectrogram.std()
    return spectrogram

In [31]:
# Load the trained model
model_path = 'best_model_state_dict.pth'
model = CustomSTTModel2(len(combined_vocabulary_list))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

CustomSTTModel2(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gru1): GRU(2048, 128, batch_first=True, bidirectional=True)
  (gru2): GRU(256, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=127, bias=True)
)

In [32]:
# Audio transformation
spectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [33]:
# Inference function
def infer(audio_path, model, transform, processor):
    spectrogram = transform_audio_to_spectrogram(audio_path, transform)
    spectrogram = spectrogram.unsqueeze(0).to(device)  # Add batch dimension and send to device

    with torch.no_grad():
        outputs = model(spectrogram)
        outputs = torch.nn.functional.log_softmax(outputs, dim=2)
        outputs = outputs.permute(1, 0, 2)  # (T, N, C)

    decoded_preds = torch.argmax(outputs, dim=2)
    decoded_preds = decoded_preds.transpose(0, 1)  # (N, T)
    # Convert output to text using CustomProcessor
    text_output = []
    decoded_preds_list = decoded_preds.flatten().tolist()
    # print(decoded_preds_list)
    pred_text = processor.int_to_text([i for i in decoded_preds_list if i < len(processor.vocab)])

    return pred_text

In [37]:
object_voc = CustomProcessor(combined_vocabulary_list)
# Example usage
audio_file = 'r3.wav'
# audio_file = dataset[0]["audio"]["path"]
predicted_text = infer(audio_file, model, spectrogram_transform, object_voc)
print(predicted_text)

sنuىييsۗللل,ييﺃ!uh
